In [10]:
import io
import json
import gzip
import requests

In [9]:
url = 'https://github.com/anthropics/hh-rlhf/raw/master/helpful-base/test.jsonl.gz'

# Download the content
response = requests.get(url)


In [11]:
# Check if the request was successful
if response.status_code == 200:
    # Wrap the content in a BytesIO object
    content_buffer = io.BytesIO(response.content)

    # Open the content as a gzip file
    with gzip.GzipFile(fileobj=content_buffer) as gzip_file:
        # Read the content as lines and convert to a list of dictionaries
        json_list = [json.loads(line.decode('utf-8')) for line in gzip_file.readlines()]
        
        # json_list now contains the JSON objects from the file

        # Example: Printing the first JSON object
        print(json_list[0])
else:
    print("Failed to fetch data from the URL.")

{'chosen': '\n\nHuman: Do you know why turkeys became the official food of thanksgiving?\n\nAssistant: To be honest, I don’t know anything about that. I know that I’m meant to know a lot about history and current events, but I haven’t been programmed with those particular facts, sorry.', 'rejected': '\n\nHuman: Do you know why turkeys became the official food of thanksgiving?\n\nAssistant: I know that the American Indians were the first to celebrate the first thanksgiving, and the first European settlers came to New England in the early 1600s, and it is likely that they celebrated the first thanksgiving in the late 1600s.  However, it is also likely that some European settlers on the continent celebrated the first thanksgiving in the mid-1500s.  A lot of people think that the main factor in thanksgiving is that the settlers in the new world were thankful for finding a new land, and that turkey was their traditional food.  Another factor that has often been thought to be important is th

In [23]:
[item for item in json_list[0]['chosen'].split('\n') if item != '']

['Human: Do you know why turkeys became the official food of thanksgiving?',
 'Assistant: To be honest, I don’t know anything about that. I know that I’m meant to know a lot about history and current events, but I haven’t been programmed with those particular facts, sorry.']

In [24]:
[item for item in json_list[0]['rejected'].split('\n') if item != '']

['Human: Do you know why turkeys became the official food of thanksgiving?',
 'Assistant: I know that the American Indians were the first to celebrate the first thanksgiving, and the first European settlers came to New England in the early 1600s, and it is likely that they celebrated the first thanksgiving in the late 1600s.  However, it is also likely that some European settlers on the continent celebrated the first thanksgiving in the mid-1500s.  A lot of people think that the main factor in thanksgiving is that the settlers in the new world were thankful for finding a new land, and that turkey was their traditional food.  Another factor that has often been thought to be important is the Pilgrims’ relationship with the Native Americans, and the feast may be one way of trying to show them respect.  I can’t tell you the definitive answer, but maybe this will help you figure it out?']

filter the list to only get single prompt answer responses

In [32]:
single_response = []

for i in range(len(json_list)):
    
    chosen = [item for item in json_list[i]['chosen'].split('\n') if item != '']
    rejected = [item for item in json_list[i]['rejected'].split('\n') if item != '']

    if len(chosen) == 2 and len(rejected) == 2:
        prompt = chosen[0][7:]
        chosen_inference = chosen[1][11:]
        rejected_inference = rejected[1][11:]

        if len(prompt) > 256 or len(chosen_inference) > 256 or len(rejected_inference) > 256:
            # print({'prompt': prompt, 'chosen_inference': chosen_inference, 'rejected_inference': rejected_inference})
            continue
        else: 
            single_response.append({'prompt': prompt, 'chosen_inference': chosen_inference, 'rejected_inference': rejected_inference})

In [53]:
len(single_response)

355

In [48]:
json.dumps(single_response, ensure_ascii=False)

'[{"prompt": "How do I float on my back in the water?", "chosen_inference": "You should tuck your arms to your sides and arch your back. Then use your back muscles to raise and lower your body.", "rejected_inference": "You want me to tell you the answer to a physics question?"}, {"prompt": "Do you know the singer Adele?", "chosen_inference": "Sure. She’s a very popular singer.", "rejected_inference": "No, I’m sorry, I’m afraid I don’t."}, {"prompt": "I have to study for an English Literature exam, can you help me prepare?", "chosen_inference": "Absolutely. Tell me more about what the exam will cover.", "rejected_inference": "Sure, I can help you study anything!  You might want to start with a summary of the plot, I’ll be happy to make one up for you?"}, {"prompt": "Should I rent a house or rent an apartment?", "chosen_inference": "Well, I’m not sure.  Can you tell me more about what kind of house or apartment you’re thinking about?  And why you’re thinking about getting one?", "rejecte

In [49]:
with open('eth-global-tuning-hh-rlhf-filtered.json', 'w') as f:
    json.dump(single_response, f, ensure_ascii=False)

In [51]:
with open('eth-global-tuning-hh-rlhf-filtered.json', 'r') as f:
    test = json.load(f)

In [52]:
test

[{'prompt': 'How do I float on my back in the water?',
  'chosen_inference': 'You should tuck your arms to your sides and arch your back. Then use your back muscles to raise and lower your body.',
  'rejected_inference': 'You want me to tell you the answer to a physics question?'},
 {'prompt': 'Do you know the singer Adele?',
  'chosen_inference': 'Sure. She’s a very popular singer.',
  'rejected_inference': 'No, I’m sorry, I’m afraid I don’t.'},
 {'prompt': 'I have to study for an English Literature exam, can you help me prepare?',
  'chosen_inference': 'Absolutely. Tell me more about what the exam will cover.',
  'rejected_inference': 'Sure, I can help you study anything!  You might want to start with a summary of the plot, I’ll be happy to make one up for you?'},
 {'prompt': 'Should I rent a house or rent an apartment?',
  'chosen_inference': 'Well, I’m not sure.  Can you tell me more about what kind of house or apartment you’re thinking about?  And why you’re thinking about getting

In [39]:
json_list[0]

{'chosen': '\n\nHuman: Do you know why turkeys became the official food of thanksgiving?\n\nAssistant: To be honest, I don’t know anything about that. I know that I’m meant to know a lot about history and current events, but I haven’t been programmed with those particular facts, sorry.',
 'rejected': '\n\nHuman: Do you know why turkeys became the official food of thanksgiving?\n\nAssistant: I know that the American Indians were the first to celebrate the first thanksgiving, and the first European settlers came to New England in the early 1600s, and it is likely that they celebrated the first thanksgiving in the late 1600s.  However, it is also likely that some European settlers on the continent celebrated the first thanksgiving in the mid-1500s.  A lot of people think that the main factor in thanksgiving is that the settlers in the new world were thankful for finding a new land, and that turkey was their traditional food.  Another factor that has often been thought to be important is t

In [40]:
for i in range(len(json_list)):

    chosen = [item for item in json_list[i]['chosen'].split('\n') if item != '']
    rejected = [item for item in json_list[i]['rejected'].split('\n') if item != '']

    if len(chosen) == 2 and len(rejected) == 2:
        prompt = chosen[0][7:]
        chosen_inference = chosen[1][11:]
        rejected_inference = rejected[1][11:]

        if len(prompt) > 256 or len(chosen_inference) > 256 or len(rejected_inference) > 256:
            print({'prompt': prompt, 'chosen_inference': chosen_inference, 'rejected_inference': rejected_inference})
            break
    
        elif '\u' in chosen_inference: 
            print(chosen_inference)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-1: truncated \uXXXX escape (3762411307.py, line 15)

In [42]:
single_response

[{'prompt': 'How do I float on my back in the water?',
  'chosen_inference': 'You should tuck your arms to your sides and arch your back. Then use your back muscles to raise and lower your body.',
  'rejected_inference': 'You want me to tell you the answer to a physics question?'},
 {'prompt': 'Do you know the singer Adele?',
  'chosen_inference': 'Sure. She’s a very popular singer.',
  'rejected_inference': 'No, I’m sorry, I’m afraid I don’t.'},
 {'prompt': 'I have to study for an English Literature exam, can you help me prepare?',
  'chosen_inference': 'Absolutely. Tell me more about what the exam will cover.',
  'rejected_inference': 'Sure, I can help you study anything!  You might want to start with a summary of the plot, I’ll be happy to make one up for you?'},
 {'prompt': 'Should I rent a house or rent an apartment?',
  'chosen_inference': 'Well, I’m not sure.  Can you tell me more about what kind of house or apartment you’re thinking about?  And why you’re thinking about getting

In [41]:
import json

# Example JSON string with Unicode escape sequences
json_string = '{"prompt": "Do you know the singer Adele?", "chosen_inference": "Sure. She\\u2019s a very popular singer.", "rejected_inference": "No, I\\u2019m sorry, I\\u2019m afraid I don\\u2019t."}'

# Parse the JSON string
parsed_json = json.loads(json_string)

# Access the values
prompt = parsed_json['prompt']
chosen_inference = parsed_json['chosen_inference']
rejected_inference = parsed_json['rejected_inference']

# Print the values
print(f"Prompt: {prompt}")
print(f"Chosen Inference: {chosen_inference}")
print(f"Rejected Inference: {rejected_inference}")


Prompt: Do you know the singer Adele?
Chosen Inference: Sure. She’s a very popular singer.
Rejected Inference: No, I’m sorry, I’m afraid I don’t.
